In [ ]:
#! pip install scipy==1.1.0
! pip install plotly --upgrade

     |████████████████████████████████| 13.2MB 317kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import sys
import os
prefix = '/content/gdrive/My Drive/'
customized_path = 'Advanced_ML/project'
sys_path = os.path.join(prefix, customized_path)
sys.path.append(sys_path)

In [ ]:
import pandas as pd
from numpy import array
from sklearn import preprocessing
df=pd.read_excel('/content/gdrive/My Drive/Advanced_ML/project/italy_data.xlsx')
df_required=df.drop(columns=['SNo','Country','RegionName','Latitude','Longitude','TotalHospitalizedPatients','CurrentPositiveCases','TotalPositiveCases','TestsPerformed'])





In [ ]:
def regions(region_code) :
  regions_list=[]
  regions_dates=[]
  regions_anamoly=[]
  for i in region_code :  
    df_region_1=df_required
    df_region_1=df_required.loc[df['RegionCode']==i]
    df_region_1=df_region_1[10:150]
    df_region_1=df_region_1.drop(columns=['RegionCode'])
    df_region_1=df_region_1.fillna(0)
    df_region_1_dates=df_region_1
    column_final=df_region_1_dates.columns.to_list()
    df_region_1_anamoly=array(df_region_1)
    df_region_1=df_region_1.drop(columns=['Date'])
    column=df_region_1.columns.to_list()
    min_max_scaler = preprocessing.MinMaxScaler()
    try :
      x_scaled = min_max_scaler.fit_transform(df_region_1)
    except :
      continue;
    df_region_1=pd.DataFrame(x_scaled,columns=column)
    D_in=df_region_1.shape[1]
    regions_list.append(df_region_1)
    regions_dates.append(df_region_1_dates)
    regions_anamoly.append(df_region_1_anamoly)
  return regions_list,regions_dates,regions_anamoly,region_code,column_final
    

In [ ]:
regions_list,regions_dates,regions_anamoly,train,column=regions([1,2,3,5,6,7,8,9,10,11,13,14,16,17,18,20,21,22])


In [ ]:
regions_list

[     HospitalizedPatients  IntensiveCarePatients  ...  Recovered    Deaths
 0                0.000000               0.024609  ...   0.000000  0.000000
 1                0.003989               0.053691  ...   0.000000  0.000485
 2                0.019088               0.071588  ...   0.000000  0.000728
 3                0.057550               0.087248  ...   0.000000  0.000728
 4                0.050997               0.098434  ...   0.000000  0.002669
 ..                    ...                    ...  ...        ...       ...
 135              0.030484               0.000000  ...   0.997035  0.999029
 136              0.031054               0.000000  ...   0.997523  0.999029
 137              0.030484               0.000000  ...   0.997673  0.999272
 138              0.028775               0.000000  ...   0.998611  1.000000
 139              0.029345               0.000000  ...   1.000000  1.000000
 
 [140 rows x 6 columns],
      HospitalizedPatients  IntensiveCarePatients  ...  Recov

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    
    def __init__(self, D_in, D_out):
        super(CNN, self).__init__()
        self.layer1=nn.Sequential(nn.Conv1d(D_in,20,kernel_size=3,padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool1d(2,padding=1)     
        )
        self.layer2=nn.Sequential(nn.Conv1d(20,40,kernel_size=3,padding=1),
                                  nn.ReLU(),
                                  nn.MaxPool1d(2,padding=1)
            
        )
        self.linear_out=nn.Linear(40*3,D_out)
    def forward(self, x):
        output=self.layer1(x)
        output=self.layer2(output)
        output = output.view(output.size(0),-1)
        y_predicted=self.linear_out(output)
        return y_predicted

In [ ]:
from numpy import array
def Data_loader(df,n_input,n_output=1) :  
    in_start=0
    X,y=list(),list()
    df=array(df)
    #df = df.reshape((df.shape[0]*df.shape[1], df.shape[2]))
    for _ in range(len(df)):
      in_end = in_start + n_input
      out_end = in_end + n_output
		# ensure we have enough data for this instance
      if out_end <= len(df):
        X_input = df[in_start:in_end]
        #x_input = x_input.reshape((len(x_input), 1))
        X.append(X_input)
        y.append(df[in_end:out_end])
      in_start += 1
    return array(X), array(y)

In [ ]:
def predictor(regions_list) :
  import numpy as np
  import torch.optim as optim
  anamoly_list=[]
  for i in range(len(regions_list)) :
    X,y=Data_loader(regions_list[i],5)
    X_tensor = torch.from_numpy(X).float()
    y_tensor = torch.from_numpy(y).float()
    def CNN_training(Model,learn_rate):
        epoch = 2000
        X,y=X_tensor,y_tensor.squeeze()
        criterion=nn.MSELoss()
        optim = torch.optim.Adam(model.parameters(), lr=learn_rate)
        for e in range(epoch):
                y_pred = Model(X)
                loss=criterion(y_pred,y)
                optim.zero_grad()
                loss.backward()
                optim.step()
        out=y_pred
        return out  
    model=CNN(5, 6)     
    yhat=CNN_training(model,learn_rate = 1e-3)
    y_true=y_tensor.detach().numpy()
    yhat=yhat.detach().numpy()
    anamoly=[]
    for i in range(len(y_true)):
      dist = np.linalg.norm( y_true[i]- yhat[i])
      if dist > 0.05 :
        a=i
        anamoly.append(a)
    anamoly_list.append(anamoly)
  return anamoly_list

In [ ]:
anamoly_list=predictor(regions_list)

In [ ]:
def anamoly(anamoly_list) :

  first_anamoly=[]
  for i in range(len(anamoly_list)) :
    a=anamoly_list[i]
    try :  
      b=a[0]
    except :
      continue
    first_anamoly.append(b)
  train_first_anamoly=min(first_anamoly)
  anamoly_dates=[]
  train_first_anamoly_day=[]
  for i in range(len(anamoly_list)) :
    a=anamoly_list[i]
    try :
      b=a[0]
    except :
      continue
    if b == train_first_anamoly :
      anamoly_dates.append(i)
    else :
      continue
  for i in anamoly_dates :
    j=train[i]
    pick=df_required.loc[df['RegionCode']==j]
    pick=pick.drop(columns=['RegionCode'])
    train_first_anamoly_day.append(pick)
    target=train_first_anamoly_day[0][10+train_first_anamoly:10+train_first_anamoly+1]
  return target


In [ ]:
anamoly_date=anamoly(anamoly_list)
anamoly_date_train=anamoly_date['Date']
anamoly_date=anamoly_date.drop(columns=['Date'])
anamoly_date=array(anamoly_date)
anamoly_date


array([[43, 17, 46, 26,  0,  2]])

In [ ]:

def test_anamoly(test_regions_list) :
  import numpy as np
  test_regions=regions(test_regions_list)[1]
  output_dates=[]
  test_anamoly_data=[]
  for k in range(len(test_regions)) :
    df_test_region=test_regions[k]
    test_region=array(test_regions[k].drop(columns=['Date']))
    distance=[]
    for i in range(len(test_region)) :
      dist = np.linalg.norm( array(test_region[i])- anamoly_date)
      distance.append(dist)
    minimum=min(distance)
    for j in range(len(distance)) :
      if minimum == distance[j] :
        test_anamoly= test_region[j]
        #test_anamoly_data.append(test_anamoly)
        flags=j
      else :
        continue
    output=df_test_region[flags:flags+1].values
    output_dates.append(output)
  return output_dates


In [ ]:
result=test_anamoly([1,12,15,19])
result


[array([['2020-03-05T17:00:00', 43, 17, 46, 26, 0, 2]], dtype=object),
 array([['2020-03-10T18:00:00', 50, 15, 34, 14, 11, 6]], dtype=object),
 array([['2020-03-09T18:00:00', 42, 8, 69, 19, 1, 0]], dtype=object),
 array([['2020-03-11T17:00:00', 23, 1, 57, 21, 2, 0]], dtype=object)]

In [ ]:

df_1=pd.DataFrame(result[1],columns=column)
df_2=pd.DataFrame(result[2],columns=column)
df_3=pd.DataFrame(result[3],columns=column)
df_4=pd.DataFrame(result[0],columns=column)

In [ ]:
df_4['Date'].values[0]

'2020-03-05T17:00:00'

In [ ]:
import plotly.express as px
df_region_12=regions([12])[1]
df_region_12=df_region_12[0][['Date','NewPositiveCases']]
fig = px.line(df_region_12, x='Date', y='NewPositiveCases',title='Lazio')
fig.update_layout(title={'text': "Lazio",
        'y':0.9,
        'x':0.5,'xanchor': 'center',
        'yanchor': 'top'})
fig.add_vline(x=df_1['Date'].values[0],line_width=3, line_dash="dash", line_color="green",)
fig.add_hline(y=df_1['NewPositiveCases'].values[0],line_width=1, line_dash="dash", line_color="red")
fig.add_annotation(x=df_1['Date'].values[0], y=df_1['NewPositiveCases'].values[0],
            text="Predicted outbreak March 10  2020, New Cases 14 ",
            showarrow=True,
            arrowhead=2,arrowsize=1,
        arrowwidth=2,)
fig.show()

In [ ]:
import plotly.express as px
df_region_15=regions([15])[1]
df_region_15=df_region_15[0][['Date','NewPositiveCases']]
fig = px.line(df_region_15, x='Date', y='NewPositiveCases',title='Campania')
fig.update_layout(title={'text': "Campania",
        'y':0.9,
        'x':0.5,'xanchor': 'center',
        'yanchor': 'top'})
fig.add_vline(x=df_2['Date'].values[0],line_width=3, line_dash="dash", line_color="green",)
fig.add_hline(y=df_2['NewPositiveCases'].values[0],line_width=1, line_dash="dash", line_color="red")
fig.add_annotation(x=df_2['Date'].values[0], y=df_2['NewPositiveCases'].values[0],
            text="Predicted outbreak March 9 2020, New Cases 19",
            showarrow=True,
            arrowhead=2,arrowsize=1,
        arrowwidth=2,)
fig.show()

In [ ]:
import plotly.express as px
df_region_19=regions([19])[1]
df_region_19=df_region_19[0][['Date','NewPositiveCases']]
fig = px.line(df_region_19, x='Date', y='NewPositiveCases',title='Sicilia')
fig.update_layout(title={'text': "Sicilia",
        'y':0.9,
        'x':0.5,'xanchor': 'center',
        'yanchor': 'top'})
fig.add_vline(x=df_3['Date'].values[0],line_width=3, line_dash="dash", line_color="green",)
fig.add_hline(y=df_3['NewPositiveCases'].values[0],line_width=1, line_dash="dash", line_color="red")
fig.add_annotation(x=df_3['Date'].values[0], y=df_3['NewPositiveCases'].values[0],
            text="Predicted outbreak March 11 2020, New Cases 21",
            showarrow=True,
            arrowhead=2,arrowsize=1,
        arrowwidth=2,)
fig.show()

In [ ]:
def PeakRatio(regions_list) :
  ratio_list=[]
  for i in regions_list :
    df_region_i=regions([i])[1]
    df_region_i_pc=df_region_i[0]['NewPositiveCases'].values.tolist()
    region_i_peak=max(df_region_i_pc)
    results=test_anamoly([i])
    predicted_peak=results[0][0][4]
    ratio=region_i_peak/predicted_peak
    ratio_list.append(ratio)
  return ratio_list 

In [ ]:
ratio_list=PeakRatio([1,12,15,19])  
ratio_list

[38.30769230769231, 15.0, 11.842105263157896, 8.095238095238095]